#Luiz Carlos Ferreira Carvalho
#DRE: 120025788

***IMPORTS/CACHE (PRECISA RODAR ANTES)***

In [2]:
#@title
import numpy as np
from fractions import Fraction
from functools import lru_cache, wraps
from random import randint

#Código para cachear função e otimizar a performance
def cache(f):

    def g(*args):
        if args not in g.cache:
            g.cache[args] = f(*args)
        return g.cache[args]
    g.cache = {}
    g.__doc__  = f.__doc__
    g.__name__ = f.__name__
    return g

#Código para cachear função que recebe um array e otimizar a performance
def np_cache(function):
    @lru_cache()
    def cached_wrapper(hashable_array):
        array = np.array(hashable_array)
        return function(array)

    @wraps(function)
    def wrapper(array):
        return cached_wrapper(tuple(array))

    wrapper.cache_info = cached_wrapper.cache_info
    wrapper.cache_clear = cached_wrapper.cache_clear

    return wrapper


**LETRA A**
Enunciado:

a) Explique como que o cenário acima descrito pode ser modelado por uma cadeia de Markov, descrevendo detalhadamente os estados de sua cadeia e exibindo a sua matriz de probabilidades de transiçãao. Cada passo de sua construçãao deve ser descrito de forma detalhada.

**LETRA A**
Resposta:

Uma cadeia de Markov é uma sequência de váriaveis aleatórias, $X_0, X_1, X_2, ..., X_n$ onde dizemos que cada uma delas representa um estado $i$ em um determinado tempo n, e exista uma probabilidade fixa ($P_{ij}$) de que ela esteja a seguir em outro estado $j$. Matematicamente para $i_0$,...,$i_{n-1}$,$i$, $j$, temos:

$P${$X_{n+1}=j | X_n=i,X_{n-1}=i_{n-1},..., X_1=i_1,X_0=i_0$ } = $P_{ij}$ => $P_{ij}(n) = P(X_{n+1} = x_j | X_n = x_i) $

Onde os valores $P_{ij}$, $0 ≤ i ≤ M, 0 ≤ j ≤ N $ são chamados de probabilidades de transição da cadeia de Markov satisfazendo a propriedade de que o seu somatório deva ser igual a 1 para cada linha da matriz,

$P_{ij}≥0$ ,  $\sum_{j=0}^{N} P_{ij} = 1$, $i = 0,1,...,M$

Assim temos a matriz de transição:

  $$ 
 P = \begin{bmatrix}
 P_{0,0} & P_{0,1}  & P_{0,2} & \cdots  & P_{0,N}  &  \\
 P_{1,0} & P_{1,1}  & P_{1,2} & \cdots  & P_{1,N}  &  \\
 P_{2,0} & P_{2,1}  & P_{2,2} & \cdots  & P_{2,N} \\
 \vdots  & \vdots  & \vdots   & \ddots  & \vdots  \\
 P_{M,0} & P_{M,1} & P_{M,2}  & \cdots  & P_{M,N}
\end{bmatrix}
$$

Sendo assim parece tentador e intuitivo usar uma cadeia de Markov para jogos de tabuleiros simples (Apenas um numero N de casas comuns, e um par de dados honestos) onde o próximo estado do jogador (casa do tabuleiro) só depende exclusivamente do estado atual (casa do tabuleiro) e do que ele tirar nos dados, ou seja, não leva em consideração o passado, por exemplo para um jogador no estado $X_n$ pouco importa seu estado $X_{n-1}$ ou ainda $X_{n-2}$, o que importa é que seu proximo estado depende apenas do rolar dos dados, e de seu estado atual, com essas informações saberemos para onde ele se moverá.

Podemos então modelar um jogo de tabuleiro simples com uma cadeia de Makorv, pois a casa atual do jogador será a linha da matriz ($i$) de trnasição em que ele está, e cada coluna ($j$) representando as casas que ele poderá alcançar após sua jogada atual. Nesta modelagem $P_{ij}$ será a probabilidade de ir para cada nova casa, e este valor é obtido ao analisar os casos favoraveis e possiveis das rolagens dos dados, dessa forma teremos um valor fixo para tais probabilidades, por exemplo, um jogador na casa 4 independente do tempo que se passou sempre terá a mesma probabilidade alcançar a casa 10, pois o par de dados se manterá o mesmo.

O Banco Imobiliário por sua vez não é um jogo de tabuleiro simples, mesmo que em sua versão simplificada para este projeto:

Regras do jogo:

* Tabuleiro de 40 casas
* Jogador começa na casa 0
* Jogador rola uma dupla de dados honesto de 6 faces cada e se movimenta para conforme o valor encontrado
* Quando tirar uma dupla de valores iguais (ex: 2 + 2) o jogador não se move e atira novamente o dado
* Tirando 3 duplas de valores iguais nos dados (ex: 2+2, 1+1, 3+3) o jogador estará preso tendo que ir para a casa 20 do tabuleiro, está representando a prisão
* Para sair da prisão o jogador precisa tirar uma duplas de valores iguais ("A esma sorte que o colocou, o tirará")
* Após 3 insucessos o jogador sairá automaticamente da prisão continunando na casa 20, porém agora no estado "Visita a Prisão"
* O jogador ao sair da prisão rola novamente os dados para de fato andar
* Se o jogador cair na cassa 20 após uma rolagem de dados distintos, este não irá preso, e sim estará visitando a prisão

Assim podemos ver que o próximo estado do jogador depende do valor dos dados da rodada anterior, isto se deve a regra da prisão, já que um jogador que tira dados com valores iguais, ou seja, 2 + 2 não altera seu estado, tendo que rolar novamente os dados, e tirando mais uma dupla repetida, este poderá ser preso no próximo turno $n$ sendo mandando diretamente pra casa de numero 20, esta regra a principio impediria a modelagem da cadeia de Markov pois estamos dependendo de eventos que ocorreram no passado $X_{n-1}$ ou ainda $X_{n-2}$, para saber a próxima casa do jogador $X_{n+1}$.

Para contornar tal problema optei por uma modelagem da cadeia de Markov com uma matriz de transição 123x123, com isso podemos registrar os estados do jogador quando este tira uma dupla de dados e quando está preso, dependendo apenas de seu estado $X_n$ para saber seu possivel próximo estado. Para representar os estados em que o jogador tirou uma dupla de dados, os chamaremos de estados auxiliares e usaremos um apóstrofo $'$ referenciado de "linha" para diferencia-los, na matriz de transição serão as linhas de 40 a 79, enquanto os estados auxiliares para quando o jogador tirar duas duplas de dados seguidas serão as linhas de 80 a 119, e usaremos duas linhas $''$ para representa-los, os estado emq ue o jogador está preso serão as linhas de 120 a 122, e usaremos um $p$ para indicar que está preso, e "linhas" para indicar a quanto tempo. Esta modelagem garante que teremos nas colunas de 40 a 119 as probabilidades do jogador tirar uma dupla de dados iguais, assim como de 120 a 122 as probabilidades dele ir preso e continuar preso.

Com esta modelagem também podemos encontrar facilmente a casa do jogador no tabuleiro usando módulo, pois se o jogador na casa 32 tirou uma dupla de dados iguais indo para o estado 72, sabemos que $32 = 72 mod(40)$ ou seja $n = n' mod(40) = n'' mod(40)$

Detalhes da modelagem:

* As linhas de 0 a 39 representam as casas do tabuleiro
* As linhas de 40 a 79 representam os estado auxiliares "linha" ($'$), quando o jogador tira valores iguais nos dados
* As linhas de 80 a 119 representam os estado auxiliares "duas linha" ($''$), quando o jogador no estado "uma linha" ($''$) tira valores iguais nos dados
* As linhas de 120 a 122 representam o estado de quando o jogador vai para a prisão, assim podemos diferenciar a casa 20 (visita a prisão)
* A linha 121 será indicada com "uma linha" ($p'$) que representa o estado em que o jogador preso tira dados distintos e continua preso
* A linha 122 será indicada com "duas linha" ($p''$) que representa o estado em que o jogador preso tira dados distintos e continua preso pela segunda vez
* As colunas de 0 a 122 seguem a mesma ideia das linhas, representando os possiveis estado possiveis do jogador dependendo apenas de seu estado atual

Assim teremos a seguinte matriz de transição:

  $$ 
 P = \begin{bmatrix}
 P_{0,0} & P_{0,1}  & P_{0,2} & \cdots  & P_{0,122}  &  \\
 P_{1,0} & P_{1,1}  & P_{1,2} & \cdots  & P_{1,122}  &  \\
 P_{2,0} & P_{2,1}  & P_{2,2} & \cdots  & P_{2,122} \\
 \vdots  & \vdots  & \vdots   & \ddots  & \vdots  \\
 P_{122,0} & P_{122,1} & P_{122,2}  & \cdots  & P_{122,122}
\end{bmatrix}
$$

Onde cada $P_{ij}$ representa a probabilidade do jogador assumir o próximo estado.

Probabilidades:

Como temos dois dados honestos de 6 faces cada, isso significa que temos 6 x 6 = 36 casos possiveis de resultados na rolagem dos dados. São eles: $(1,1)$, $(1,2)$, $(1,3)$,...,$(6,5)$,$(6,6)$.

Os valores possiveis de encontrarmos após somar os dados são representados nos eguinte vetor:

$$\text{V = [2,3,4,5,6,7,8,9,10,11,12]}$$

Para encontrarmos a probabilidade de cada soma dos valores dos dados usamos:

$\frac{Casos favoraveis}{Casos possiveis}$

Logo nosso vetor de probabilidades dos dados é:

$$\text{V = [2,3,4,5,6,7,8,9,10,11,12]} → [\frac{1}{36},\frac{1}{18}, \frac{1}{12}, \frac{1}{9}, \frac{5}{36}, \frac{1}{6}, \frac{5}{36}, \frac{1}{9}, \frac{1}{12}, \frac{1}{18}, \frac{1}{36}]$$

A soma dos valores é quantidade casas no tabuleiro que o jogador anda, assim a probabilidade na matriz de transição da cadeia de Markov dele mudar de um estado $X_n$ para $X_{n+1}$ será as probabilidades acima distribuidas pelas coluna da matriz, por exemplo um jogador na casa 12 ($i = 12$), tem 0 probabilidade de ir para a casa 13 ($j=13$) ou qualquer casa além da 24 ($j=24$), já que a soma dos dados vai de 2 a 12, porém ele pode ir para a casa 14 ($j=14$) se tirar 1 + 1 nos dados, neste caso a probabilidade ($P_{12,14}$) será $\frac{1}{36}$.

Porém no caso do banco imobiliário quando o jogador tirar uma dupla de valores iguais nos dados, ele não irá se mover terá que jogar novamente os dados, assim a probabilidade dele ir para a casa 14, estando na casa 12 é 0. Assim se o jogador tirar qualquer duplas de valores iguais nos dados ele irá continuar na casa 12 tendo que jogar novamente os dados, em nossa modelagem o jogador ao tirar uma dupla irá para o estado "linha", neste caso o jogador no estado 12 ao tirar 1 + 1 nos dados irá para o estado 52 ou $12'$, como ele tem 6 casos favoraveis $(1,1),(2,2),(3,3),(4,4),(5,5),(6,6)$ de tirar uma duplas de valores iguais, ele tem probabilidade ($P_{12,52}$) $\frac{6}{36}$ ou $\frac{1}{6}$ de ir para o estado 52.

Assim em nossa modelagem, ao removermos as probabilidades de valores iguais dos dados e incluindo a probabilidade de obtermos 0 e 1, encontramos o seguinte vetor de valores e probabilidades:

$$\text{V = [0,1,2,3,4,5,6,7,8,9,10,11,12]} → [0,0,0,\frac{1}{18}, \frac{1}{18}, \frac{1}{9}, \frac{1}{9}, \frac{1}{6}, \frac{1}{9}, \frac{1}{9}, \frac{1}{18}, \frac{1}{18}, 0]$$

Portanto em cada linha de 0 ($i=0$) a 119 ($i=119$) de nossa matriz de transição o jogador terá, as probabilidades acima (distribuidas conforma a soma do estado atual modulo 40 com o index do vetor de probabilidades) de mudar para um dos estados das colunas de 0 ($j=0$) a 39 ($j=39$), e adicionando a probabilidade $\frac{1}{6}$ no estado "linha" referente aquela linha, se linha 14 ($i=14$), estado 54 ($j=54$) ($14'$) tera probabilidade $P_{14,54}$ = $\frac{1}{6}$.

OBS: Caso o jogador possa dar a volta no tabuleiro no estado em que ele está atualmente a probabilidade que exceder o tabuleiro sera colocada no inicio da linha, exemplo, o jogar na casa 35 se tirar 4 + 5 nos dados terá completado uma volta no tabuleiro, e irá para a casa 4, assim a probabilidade $P_{35,4} = \frac{1}{9}$. Para popular o inicio da linha basta somar o indice do vetor de probabilidades com a linha atual modulo 40.

Nos casos das linhas de 80 ($i=80$) a 119 ($i=119$) colocamos a probabilidade $\frac{1}{6}$ no estado 120 ($j=120$), pois este representa a prisão, e um jogador no estado "duas linhas" ao tirar uma dupla de valores iguais estará preso.

Nos casos especiais de 120 ($i=120$) e 121 ($i=121$), o jogador terá probabilidade $\frac{5}{6}$ ao rolar uma duplas de valores distintos nos dados, de continuar preso sendo esta colocada nas colunas 121 ($j=121$) e 122 ($j=122$)respectivamente para cada estado e probabilidade $\frac{1}{6}$ de voltar para a casa 20 ($j=20$) ao rolar uma dupla de valores iguais nos dados. Já o caso 122 ($i=122$) colocamos probabilidade 1 na coluna 20 ($j=10$), já que o jogador será obrigatoriamente solto neste estado.

Exemplos:

$$
  M_{123x123} = \begin{bmatrix}
 0 & 0  & 0 & 1/18 & 1/18 &  \cdots & 1/6 &  \cdots & 0  &  \\
 0 & 0  & 0 & 0 & 1/18 &\cdots  & 0 &\cdots  & 0  &  \\
 0 & 0  & 0 & 0 & 0 & \cdots  & 0 &\cdots  & 0 \\
 \vdots  & \vdots  & \vdots   & \vdots  & \vdots  & \cdots  & \vdots & \ddots & \vdots  \\
 0 & 0 & 0 & 0 & 0  & \cdots  & 0 & 0 & 5/6 \\
 0 & 0 & 0 & 0 & 0  & \cdots  & 0 & 0 & 0
  \end{bmatrix}
$$ 

Abaixo segue o codigo para imprimir cada linha da matriz de transição:

In [3]:
#@title
@cache
def linhaMatrizTransicao(linha):

    #Linha da matriz de transicao com 123 casas
    linhaMarkov = np.zeros([123])

    #Vetor de probabilidades da soma dos valores dos dados com inclusão do 0 e do 1
    probsCasas = [0, 0, 0, 1/18, 1/18, 1/9, 1/9, 1/6, 1/9, 1/9, 1/18, 1/18, 0]

    #Primeiro verificamos se a linha (estado) desejado é um caso de prisão ou não
    if (linha <= 119):

        #Fazemos uma variavel auxiliar para obter o valor do estado módulo 40 e 
        #saber qual casa de 0 a 39 ele correspodne de fato
        linhaAux = linha % 40
        
        #Percorremos o nosso vetor de probabilidades para preencher a linha da
        #matriz de transicao
        for i, p in enumerate(probsCasas):

            #Caso ele possa completar uma volta nessa rodada, aplicamos mod 40
            #para voltar ao inicio da linha
            if (i + linhaAux > 39):
                linhaMarkov[(linhaAux+i)%40] = p
            else:
                #Caso contrario apenas associamos a coluna da linha a probabilidade
                #Para encontrar a coluna fazemos estado atual mod 40(linhaAux) + i (indice vetor probabilidades)
                linhaMarkov[linhaAux+i] = p

        #Verificamos se o jogador está no estado normal ou no caso uma linha
        if (linha < 80):
            #Caso esteja associamos probabilidade 1/6 dele tirar uma dupla
            #de valores iguais nos dados ao estado atual + 40 que sera o 
            #proximo estado auxiliar
            linhaMarkov[linha + 40] = 1/6

        #Verificamos se o jogador está no estado duas linhas
        if (linha >= 80 and linha <= 119):
            #Caso esteja associamos probabilidade 1/6 dele tirar uma dupla
            #de valores iguais nos dados e ir preso para o estado 120
            linhaMarkov[120] = 1/6

    else:
        #Verificamos se o jogador está no primeiro estado prisao
        if (linha == 120):
            #Adicionamos probabilidade 1/6 dele tirar valores iguais nos dados
            #e ser solto indo pro estado 20
            linhaMarkov[20] = 1/6
            #Adicionamos probabilidade 5/6 dele tirar valores diferentes nos dados
            #e continuar preso indo para o estado 121
            linhaMarkov[121] = 5/6

        #Verificamos se o jogador está no segundo estado prisao
        #que se comporta de forma parecida com o anterior
        if (linha == 121):
            linhaMarkov[20] = 1/6
            linhaMarkov[122] = 5/6

        #Verificamos se o jogador está no terceiro estado prisao
        if (linha == 122):
            # Adicionamos probabilidade 1 do jogador ser solto e ir para o estado 20
            linhaMarkov[20] = 1
        
    return linhaMarkov

#Funcao que imprime linha de uma matriz com valores decimais em fracao, obrigado stack overflow
def imprimeLinhaMatriz(estado):
    linha = linhaMatrizTransicao(estado)

    linhaAux = []
    for j in range(len(linha)):
        linhaAux.append(str(Fraction(linha[j]).limit_denominator()))

    print('Linha do estado ' + str(estado))
    print(''.join(['{:6}'.format('' + str(item) + '  ') for item in range(0, 123)]))
    print(''.join(['{:6}'.format(item) for item in linhaAux]))

#Exemplos
print("\n")
imprimeLinhaMatriz(11)
print("\n")
imprimeLinhaMatriz(51)
print("\n")
imprimeLinhaMatriz(91)
print("\n")
imprimeLinhaMatriz(120)
print("\n")
imprimeLinhaMatriz(121)
print("\n")
imprimeLinhaMatriz(122)



Linha do estado 11
0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18    19    20    21    22    23    24    25    26    27    28    29    30    31    32    33    34    35    36    37    38    39    40    41    42    43    44    45    46    47    48    49    50    51    52    53    54    55    56    57    58    59    60    61    62    63    64    65    66    67    68    69    70    71    72    73    74    75    76    77    78    79    80    81    82    83    84    85    86    87    88    89    90    91    92    93    94    95    96    97    98    99    100   101   102   103   104   105   106   107   108   109   110   111   112   113   114   115   116   117   118   119   120   121   122   
0     0     0     0     0     0     0     0     0     0     0     0     0     0     1/18  1/18  1/9   1/9   1/6   1/9   1/9   1/18  1/18  0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     

Caso queira imprimir a matriz de transição inteira rode o código abaixo

In [4]:
#@title
def imprimeMatriz(m):
  auxM = []
  for i in range(len(m)):
    aux = []
    for j in range(len(m[i])):
      s = str(Fraction(m[i][j]).limit_denominator())
      aux.append(s)
    auxM.append(aux)
  
  print(''.join(['{:6}'.format('' + str(item) + '  ') for item in range(0, 123)]))
  print(('\n\n').join([''.join(['{:5}'.format(item) for item in row]) for row in auxM]))

matrizTransicao = np.zeros([123, 123])

for i in range(123):
    matrizTransicao[i] = linhaMatrizTransicao(i)

imprimeMatriz(matrizTransicao)

0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18    19    20    21    22    23    24    25    26    27    28    29    30    31    32    33    34    35    36    37    38    39    40    41    42    43    44    45    46    47    48    49    50    51    52    53    54    55    56    57    58    59    60    61    62    63    64    65    66    67    68    69    70    71    72    73    74    75    76    77    78    79    80    81    82    83    84    85    86    87    88    89    90    91    92    93    94    95    96    97    98    99    100   101   102   103   104   105   106   107   108   109   110   111   112   113   114   115   116   117   118   119   120   121   122   
0    0    0    1/18 1/18 1/9  1/9  1/6  1/9  1/9  1/18 1/18 0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    1/6  0    0    0    0    0    0    0    0    0    0    0    0

**LETRA B**
Enunciado:

b) Denote por (Xt)t∈Z ≥ 0 a posição do jogador no tabuleiro ao longo do tempo. Pelo item a), sabemos que essa sequência de variáveis aleatórias é descrita por uma cadeia de Markov. Construa um algoritmo para simular observações de tal cadeia, onde é possível controlar o número de iterações desejadas.

**LETRA B**
Resposta:

Para responder essa questão foi feito um algoritmo que usa a matriz de transição para obter o próximo estado do jogador.

Adotamos, o estado inicial como 0, porém pode ser passado algum outro estado desejado, para o inicio do algoritmo como um argumento opcional, com a matriz de transição sabemos exatamente quais são os possiveis estados futuros ($X_{n+1}$) de um jogador no estado atual $X_n$, assim se pegarmos a linha referente ao estado atual eliminarmos os estado cuja probabilidade é 0, nos resto os estados possiveis para onde o jogador pode se mover.

Para isso criei um dicionario que associa o estado $j$ do jogador a sua probabilidade $P_{ij}$, após isso populo um vetor de tamanho 36, com esses estados $j$ de acordo com as suas probabilidades, por exemplo, o jogador no estado $i=0$ possui probabilidade $P_{0,3} = \frac{1}{18}$ logo coloco o estado $j=3$ duas vezes no vetor de tamanho 36. Após fazer para todos os $j's$ daquela linha sorteio um valor desse vetor, usando a função randint do python, este valor sorteado será o novo estado do jogador.

Faço isso $n$ vezes de acordo com a entrada do jogador, e ao final retorno a lista dos estados que o jogador percorreu nesta simulação, para melhor visualização, criei um vetor auxiliar especifico para exibição que é impresso por padrão, e nele caso o estado seja maior do que 39 aplico mod 40 para encontrar a casa em que o jogador estaria parado e adiciono $'$ a string para melhor visualização assim o estado $53$ é igual a $13'$ e o $93$ é igual a $13''$.

Para os estados de prisão coloco um $p$ e as "linhas" para indicar que era a prisão.

Infelizmente usando a linha da matriz não posso indicar o resultado individual da rolagem dos dados, já que não a faço de fato, porém é possivel saber a soma dos dados ao fazer $X_n - X_{n+1}$.

OBS: Atribui a função de simulação para uma variavel afim do colab não printar o return que será usado na letra C, o print do vetor de exibição está dentro da função simulação, por padrão é setado como True e pode ser desativado como uma variavel, opcional, assim como o estado inicial do jogador.

In [5]:
#@title
#Esta função retorna o vetor dos estados possiveis que um jogador pode assumir
#dado o estado atual como parametro da função
@cache
def probablidades(estado:int):

    #Retorna a linha da matriz de transicao daquele estado
    linhaMatriz = linhaMatrizTransicao(estado)

    #Cria dicionario 
    estadosPossiveis = dict()

    #Armazena no dicionario os estado possiveis, ou seja, aquele onde a probabilidade
    #é diferente de zero, onde as chaves são o estado j, o valor a probabilidade
    for i, p in enumerate(linhaMatriz):
        if (p != 0):
            estadosPossiveis[i] = p

    estadosPossiveisComPeso = []

    #Popula o vetor dos estados possiveis aplicando um peso de acordo com a 
    #probabilidade de cada estado
    for chave in estadosPossiveis:
        #Transforma porbabilidade em fração
        probabilidade = Fraction(estadosPossiveis.get(chave)).limit_denominator()
        k = probabilidade.numerator
        #Verifica se fração possui 36 no denominador
        if (probabilidade.denominator != 36):
            #Caso não possua transforma o numerador, para antes da simplificação
            k = (36//probabilidade.denominator) * probabilidade.numerator
        #Adiciona no vetor o estado o numero de vezes igual ao numerador da probabilidade
        estadosPossiveisComPeso.extend([chave]*k)

    return estadosPossiveisComPeso

#Esta função apenas sorteia o proximo estado usando randint
#Foi separada para poder ser usado cache na outra função melhorando a performance
def proximoEstado(estado:int):
    
    probs = probablidades(estado)

    novoEstado = probs[randint(0, 35)]

    return novoEstado

#Função que executa a simulação do jogo
def simulacao(t:int, imprimir=True, estadoInicial=0):
    #Inicia os vetores de jogadas e de exibição usando o estado inicial
    jogadas = [estadoInicial]
    jogadasExibicao=[str(estadoInicial)]

    #Faz um while para contemplar o numero de jogadas passadas em t
    contador = 1
    while contador < t:
        #Determina o proximo estado do jogador usando o estado atual
        jogada = proximoEstado(jogadas[contador-1])
        #Verifica se deve imprimir o vetor de exibicão
        if (imprimir == True):
          #Caso deva imprimir começa a logica para popula-lo, verificando se é um estado de prisao
          if (jogada > 119):
              #Caso seja um estado de prisao é usado o 20p para indicar que esta preso, e o linha
              #para indicar que esta mais de um turno preso ou nao
              jogadasExibicao.append("20p" + ((jogada % 40) * "'") )
          else:
              #Caso nao esteja preso usa o linha e modulo 40 para indicar os estado auxiliares
              jogadasExibicao.append(str(jogada%40) + ((jogada // 40) * "'") )
        jogadas.append(jogada)
        contador += 1

    #Verifica se deve imprimir o vetor de exibicão
    if (imprimir == True):
        print(jogadasExibicao)

    return jogadas

s = simulacao(10000)

['0', '10', '16', "16'", '21', '28', '37', '4', '11', '15', "15'", '26', "26'", '32', '3', '8', "8'", '14', '25', '28', '38', "38'", '6', "6'", '15', '22', '33', '2', '9', '12', '15', '20', '27', '33', "33'", '36', '2', '9', "9'", '17', '22', '31', '38', "38'", '3', '10', '15', '19', "19'", '28', '33', '0', '6', '14', '21', '27', "27'", '35', '4', '10', '17', '28', '33', '2', '9', '15', "15'", '24', '34', '1', '10', "10'", '21', '29', '34', '39', '9', '16', '23', "23'", '30', '36', "36'", '3', '13', "13'", '19', '27', '35', '1', '8', "8'", '16', '27', '34', '1', '7', '12', "12'", "12''", '20', '28', '33', '38', '5', '16', '21', '31', '38', '6', '9', '14', '20', '31', '2', "2'", '6', '12', "12'", '21', '27', '37', "37'", "37''", '6', '12', '19', '22', '32', '0', '6', '13', '17', '23', '29', "29'", '38', '5', "5'", '12', '17', '26', '33', '1', '8', '13', "13'", '19', '30', "30'", '36', "36'", '6', '15', '21', '30', '36', '3', '8', '15', "15'", "15''", '24', "24'", "24''", '33', '37', '4'

**LETRA C**
Enunciado:

c) Seja Y a variável aleatória que representa a quantidade de passos que o jogador dá até a sua primeira prisão (tome cuidado, não é a primeira “visita à prisão”!). Obtenha uma aproximação para o valor esperado de Y, através de sucessivas simulações da cadeia usando o algoritmo construído no item b). Descreva detalhadamente o seu procedimento para realizar tal aproximação.

**LETRA C**
Resposta:

In [ ]:
#@title
@np_cache
def passosAtePrisao(simulacao):

    contador = 0
    preso = False

    for j in simulacao:
        if (j == 120):
            preso = True
            break
        elif (j < 40):
            contador += 1

    return contador if preso == True else 0

def media(t, imprimir=True):
  y = []

  contador = 0
  while contador < t:
      s = simulacao(10000, False)
      p = passosAtePrisao(s)

      y.append(p)

      contador += 1

  if (imprimir):
    print(y)

  return sum(y)/len(y)

e = media(1000, False)
print("E(Y): " + str(round(e)))

E(Y): 215


**LETRA D**
Enunciado:

d) Justifique matematicamente porque o procedimento realizado no item c) funciona para aproximar o valor esperado de Y.

**LETRA D**
Resposta:

**LETRA E**
Enunciado:

e) Seja Z a variável aleatória que representa a posição do jogador no tabuleiro após transcorrido um tempo suficientemente longo, ou seja, Xt quando t → ∞. Encontre a função massa de probabilidade de Z. Em média, em qual casa o jogador passa mais tempo? Discuta os resultados obtidos.

**LETRA E**
Resposta:

In [ ]:
#@title
def matrizTransicao():

    matriz = np.zeros([123, 123])
    for i in range(0, 123):
        linha = linhaMatrizTransicao(i)
        matriz[i] = linha

    return matriz

def probablidadesAposTempo(t:int):
    m = matrizTransicao()

    posicao = np.zeros([123])
    posicao[0] = 1

    contador = 0
    while contador < t:
        posicao = posicao.dot(m)
        contador += 1

    probablidades = []

    for i in range(0, 123):
        z = posicao[i]

        if (i >= 40 and i < 120):
            probablidades[i%40] += z
        elif (i >= 120):
            probablidades[20] += z
        else:
            probablidades.append(z)

    return probablidades

def probabilidadePosicaoZ(z, t=1000000):
    probablidades = probablidadesAposTempo(t)

    p = probablidades[z]

    return p

def esperancaZ(probablidades):

    e = 0
    for i in range(len(probablidades)):
        e += (i*probablidades[i])

    return round(e)

#z = probPosicaoZ(20, 100000)

probabilidades = probablidadesAposTempo(10000)

print(probabilidades)

e = esperancaZ(probabilidades)

print(e)

[0.02467988208940809, 0.024664198447949908, 0.02466028499655648, 0.024648396083852938, 0.024634861825702806, 0.024620788400229982, 0.02459653201647892, 0.02457866704770447, 0.024559082261528434, 0.024544639036735195, 0.0245292317770443, 0.024515203684677855, 0.024499879332269044, 0.024483386916049617, 0.02446658711667664, 0.024449356714021302, 0.0244329499269092, 0.024416466841842493, 0.02440074670938412, 0.02438488094409046, 0.03865625995633898, 0.024352901101980726, 0.024336643802259222, 0.024624603279588837, 0.024608334449370516, 0.0248964552243575, 0.02490057616205144, 0.025209008711133567, 0.02494931856794078, 0.02499519223708893, 0.02479870238796022, 0.024871420421462103, 0.024685747011149884, 0.024737150426299724, 0.024818775239738224, 0.024799113406921484, 0.02480877074941528, 0.024755854768010246, 0.024739868266280968, 0.024689281661522646]
20
